In [1]:
import argparse
import os
import copy

import torch
from torch import nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm

from models import SRCNN
from datasets import TrainDataset, EvalDataset
from utils import AverageMeter, calc_psnr

import numpy as np

import  torchvision


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'mps')


In [3]:
vgg_16 = torchvision.models.vgg16(weights='DEFAULT').to(device)

In [4]:
def features(img, model_cut):
    im = torch.stack((img,img,img), dim=2)[0].to(device)
    return model_cut(im)

def featureLoss(model,j):
    def feature_loss(output, target):
        
        model_cut=nn.Sequential(*list(model.modules())[0][0:j])

        output_ft = features(output, model_cut)
        target_ft = features(target,model_cut)

        scale = 1 / (np.prod(output_ft.shape))
        
        return (scale * torch.norm(output_ft - target_ft))
    return feature_loss
        

In [5]:
num_workers = 8
# all the arguments in variables with their default values
train_file = 'original_training_data/x4/91-image_x4.h5' #TODO
#train_file = 'original_training_data/for_training/AMVTG_004.h5' #TODO
eval_file = 'original_training_data/x4/Set5_x4.h5' #TODO
outputs_dir = 'outputs'
scale = 4
lr = 1e-4
batch_size = 16
num_epochs = 10
num_workers = 8
seed = 123

# new output dir using the statics variables
outputs_dir = os.path.join(outputs_dir, 'x{}'.format(scale))

# if not os.path.exists(args.outputs_dir):
#     os.makedirs(args.outputs_dir)


if not os.path.exists(outputs_dir):
    os.makedirs(outputs_dir)

cudnn.benchmark = True
device = torch.device('cuda:0' if torch.cuda.is_available() else 'mps')

# torch.manual_seed(args.seed)
torch.manual_seed(seed)

model_orig = SRCNN().to(device)
model_orig.load_state_dict(torch.load("pretrained/srcnn_x4.pth", map_location=device))
model_orig.to(device)
model_orig.eval()

model_feat = SRCNN().to(device)

#criterion = nn.MSELoss()
criterion = featureLoss(list(vgg_16.children())[0],23) #This is the 3-3 ReLu block in the paper.
optimizer = optim.Adam([
    {'params': model_feat.conv1.parameters()},
    {'params': model_feat.conv2.parameters()},
    {'params': model_feat.conv3.parameters(), 'lr': lr * 0.1}
], lr=lr)

train_dataset = TrainDataset(train_file)
train_dataloader = DataLoader(dataset=train_dataset,
                                batch_size=batch_size,
                                shuffle=True,
                                num_workers=num_workers,
                                pin_memory=True,
                                drop_last=True)
eval_dataset = EvalDataset(eval_file)
eval_dataloader = DataLoader(dataset=eval_dataset, batch_size=1)

best_weights = copy.deepcopy(model_feat.state_dict())
best_epoch = 0
best_psnr = 0.0

for epoch in range(num_epochs):
    model_feat.train()
    epoch_losses = AverageMeter()

    with tqdm(total=(len(train_dataset) - len(train_dataset) % batch_size)) as t:
        t.set_description('epoch: {}/{}'.format(epoch, num_epochs - 1))

        for data in train_dataloader:
            inputs, labels = data

            inputs = inputs.to(device)
            labels = labels.to(device)

            preds = model_feat(inputs)

            loss = criterion(preds, labels)

            epoch_losses.update(loss.item(), len(inputs))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            t.set_postfix(loss='{:.6f}'.format(epoch_losses.avg))
            t.update(len(inputs))

    torch.save(model_feat.state_dict(), os.path.join(outputs_dir, 'epoch_{}.pth'.format(epoch)))

    model_feat.eval()
    epoch_psnr = AverageMeter()

    for data in eval_dataloader:
        inputs, labels = data

        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            preds = model_feat(inputs).clamp(0.0, 1.0)

        epoch_psnr.update(calc_psnr(preds, labels), len(inputs))

    print('eval psnr: {:.2f}'.format(epoch_psnr.avg))

    if epoch_psnr.avg > best_psnr:
        best_epoch = epoch
        best_psnr = epoch_psnr.avg
        best_weights = copy.deepcopy(model_feat.state_dict())

print('best epoch: {}, psnr: {:.2f}'.format(best_epoch, best_psnr))
torch.save(best_weights, os.path.join(outputs_dir, 'best.pth'))


epoch: 0/9: 100%|██████████| 21760/21760 [01:26<00:00, 252.56it/s, loss=0.011536]


eval psnr: 17.21


epoch: 1/9: 100%|██████████| 21760/21760 [01:24<00:00, 258.24it/s, loss=0.010442]


eval psnr: 19.06


epoch: 2/9: 100%|██████████| 21760/21760 [01:23<00:00, 261.29it/s, loss=0.010245]


...

eval psnr: 21.63


epoch: 8/9: 100%|██████████| 21760/21760 [01:25<00:00, 253.09it/s, loss=0.009566]


eval psnr: 19.14


epoch: 9/9: 100%|██████████| 21760/21760 [01:25<00:00, 255.58it/s, loss=0.009503]


eval psnr: 19.40
best epoch: 3, psnr: 21.69


In [6]:
model_MSE = SRCNN().to(device)

criterion = nn.MSELoss()
#criterion = featureLoss(model_orig, 2) #this number is j
optimizer = optim.Adam([
    {'params': model_MSE.conv1.parameters()},
    {'params': model_MSE.conv2.parameters()},
    {'params': model_MSE.conv3.parameters(), 'lr': lr * 0.1}
], lr=lr)

train_dataset = TrainDataset(train_file)
train_dataloader = DataLoader(dataset=train_dataset,
                                batch_size=batch_size,
                                shuffle=True,
                                num_workers=num_workers,
                                pin_memory=True,
                                drop_last=True)
eval_dataset = EvalDataset(eval_file)
eval_dataloader = DataLoader(dataset=eval_dataset, batch_size=1)

best_weights = copy.deepcopy(model_MSE.state_dict())
best_epoch = 0
best_psnr = 0.0

for epoch in range(num_epochs):
    model_MSE.train()
    epoch_losses = AverageMeter()

    with tqdm(total=(len(train_dataset) - len(train_dataset) % batch_size)) as t:
        t.set_description('epoch: {}/{}'.format(epoch, num_epochs - 1))

        for data in train_dataloader:
            inputs, labels = data

            inputs = inputs.to(device)
            labels = labels.to(device)

            preds = model_MSE(inputs)

            loss = criterion(preds, labels)

            epoch_losses.update(loss.item(), len(inputs))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            t.set_postfix(loss='{:.6f}'.format(epoch_losses.avg))
            t.update(len(inputs))

    torch.save(model_MSE.state_dict(), os.path.join(outputs_dir, 'epoch_{}.pth'.format(epoch)))

    model_MSE.eval()
    epoch_psnr = AverageMeter()

    for data in eval_dataloader:
        inputs, labels = data

        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            preds = model_MSE(inputs).clamp(0.0, 1.0)

        epoch_psnr.update(calc_psnr(preds, labels), len(inputs))

    print('eval psnr: {:.2f}'.format(epoch_psnr.avg))

    if epoch_psnr.avg > best_psnr:
        best_epoch = epoch
        best_psnr = epoch_psnr.avg
        best_weights = copy.deepcopy(model_MSE.state_dict())

print('best epoch: {}, psnr: {:.2f}'.format(best_epoch, best_psnr))
torch.save(best_weights, os.path.join(outputs_dir, 'best.pth'))


epoch: 0/9:   0%|          | 0/21760 [00:00<?, ?it/s]

epoch: 0/9: 100%|██████████| 21760/21760 [00:56<00:00, 384.92it/s, loss=0.006608] 


eval psnr: 28.82


epoch: 1/9: 100%|██████████| 21760/21760 [00:55<00:00, 393.10it/s, loss=0.002877] 


eval psnr: 28.99


...

eval psnr: 29.31


epoch: 8/9: 100%|██████████| 21760/21760 [01:02<00:00, 350.11it/s, loss=0.002688] 


eval psnr: 29.31


epoch: 9/9: 100%|██████████| 21760/21760 [00:56<00:00, 383.44it/s, loss=0.002675] 


eval psnr: 29.36
best epoch: 9, psnr: 29.36


In [10]:
for data in eval_dataloader:
    inputs, labels = data

    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        preds_feat = model_feat(inputs).clamp(0.0, 1.0)
        preds_MSE = model_MSE(inputs).clamp(0.0, 1.0)

    print(calc_psnr(preds_feat, labels), calc_psnr(preds_MSE, labels))




tensor(20.4760, device='mps:0') tensor(33.0503, device='mps:0')
tensor(19.0577, device='mps:0') tensor(30.3439, device='mps:0')
tensor(18.1012, device='mps:0') tensor(24.4742, device='mps:0')
tensor(19.7343, device='mps:0') tensor(30.9082, device='mps:0')
tensor(19.6061, device='mps:0') tensor(28.0476, device='mps:0')


In [13]:
import argparse

import torch
import torch.backends.cudnn as cudnn
import numpy as np
import PIL.Image as pil_image

from models import SRCNN
from utils import convert_rgb_to_ycbcr, convert_ycbcr_to_rgb, calc_psnr

In [22]:
image_name = '0002.png'
path_to_outputs = f'outputs/feat/{image_name[:-4]}'


#path_to_image = 'test_images/comic.png'
path_to_image = './data/butterfly_GT.bmp'
# path_to_image = f'./videos/test_set/AMVTG_004/truth/{image_name}'

target_scale = 4

In [23]:
image = pil_image.open(path_to_image).convert('RGB')
print(type(image))

# Why are we resizing the image in this way?
image.save(path_to_outputs + '_original.png')
image_width = (image.width // target_scale) * target_scale
image_height = (image.height // target_scale) * target_scale
image = image.resize((image_width, image_height), resample=pil_image.BICUBIC)
image = image.resize((image.width // target_scale, image.height // target_scale), resample=pil_image.BICUBIC)
image = image.resize((image.width * target_scale, image.height * target_scale), resample=pil_image.BICUBIC)
image.save(path_to_outputs + '_bicubic_x{}.png'.format(target_scale))

image = np.array(image).astype(np.float32)
ycbcr = convert_rgb_to_ycbcr(image)

y = ycbcr[..., 0]
y /= 255.
y = torch.from_numpy(y).to(device)
y = y.unsqueeze(0).unsqueeze(0)

with torch.no_grad():
    preds_feat = model_feat(y).clamp(0.0, 1.0)

psnr = calc_psnr(y, preds_feat).item()
print('PSNR: {:.2f}'.format(psnr))

preds_feat = preds_feat.mul(255.0).cpu().numpy().squeeze(0).squeeze(0)

output = np.array([preds_feat, ycbcr[..., 1], ycbcr[..., 2]]).transpose([1, 2, 0])
output = np.clip(convert_ycbcr_to_rgb(output), 0.0, 255.0).astype(np.uint8)
output = pil_image.fromarray(output)
output.save(path_to_outputs + '_srcnn_x{}.png'.format(target_scale))

<class 'PIL.Image.Image'>


/var/folders/wt/brzs49rd0z900x75tbs4fzlr0000gn/T/ipykernel_57801/3953968589.py:8: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  image = image.resize((image_width, image_height), resample=pil_image.BICUBIC)
/var/folders/wt/brzs49rd0z900x75tbs4fzlr0000gn/T/ipykernel_57801/3953968589.py:9: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  image = image.resize((image.width // target_scale, image.height // target_scale), resample=pil_image.BICUBIC)
/var/folders/wt/brzs49rd0z900x75tbs4fzlr0000gn/T/ipykernel_57801/3953968589.py:10: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  image = image.resize((image.width * target_scale, image.height * target_scale), resample=pil_image.BICUBIC)


PSNR: 19.16
